In [7]:
#!pip install ortools
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [8]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [9]:
class init_loop_vrp:
    def __init__(self):
        pass
        
    def print_solution(self):
        """Prints solution on console."""
        print(f'Objective: {self.solution.ObjectiveValue()}')
        #total_time = 0
        #total_load = 0
        self.all_loop_route = []
        self.demand_route = []
        for vehicle_id in range(self.data['num_vehicles']):
            self.vehicle_route = []
            index = self.routing.Start(vehicle_id)

            while not self.routing.IsEnd(index):
                node_index = self.manager.IndexToNode(index)
                previous_index = index
                index = self.solution.Value(self.routing.NextVar(index))
                self.vehicle_route.append(node_index)

            self.all_loop_route.append(self.vehicle_route)
          
        for route in self.all_loop_route:
            self.car_demand = []
            for i in route:
                self.car_demand.append(self.data['demands'][i])

            self.demand_route.append(self.car_demand)

    def get_loop(self, num_vehicles, input_node, time_matrix):
        '''
        num_vehicles : 사용할 차량 대수 (int)
        input_node : 초기에 사용자가 신청한 노드의 index (list)
        time_matrix : 가능한 모든 노드 간의 time matrix (array)
        '''
        """Entry point of the program."""
        # Instantiate the data problem.
        self.data = {}
        self.data['num_vehicles'] = num_vehicles
        self.data['time_matrix'] = pd.DataFrame(time_matrix)[input_node].iloc[input_node].to_numpy()
        self.data['vehicle_capacities'] = [int(input()) for _ in range(num_vehicles)]
        #print(self.data['vehicle_capacities'])
        self.data['demands'] = [0,2,2] + [1 for i in range(8)]  #[0] + [sum(self.data['vehicle_capacities'])-len(time_matrix)] + [1 for i in range(len(time_matrix))] 
        self.data['depot'] = 0

        # Create the routing index manager.
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['time_matrix']), self.data['num_vehicles'], self.data['depot'])

        # Create Routing Model.
        self.routing = pywrapcp.RoutingModel(self.manager)


        # 1-1. Define cost of each arc.
        def distance_callback(from_index, to_index):
            """Returns the manhattan distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            to_node = self.manager.IndexToNode(to_index)
            return self.data['time_matrix'][from_node][to_node]

        transit_callback_index = self.routing.RegisterTransitCallback(distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # 1-2.Add Distance constraint.
        dimension_name = 'Distance'
        self.routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            30000,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)

        distance_dimension = self.routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # 2-1.Define cost of each node.
        def demand_callback(from_index):
            """Returns the demand of the node."""
            # Convert from routing variable Index to demands NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            return self.data['demands'][from_node]

        demand_callback_index = self.routing.RegisterUnaryTransitCallback(
            demand_callback)

        # 2-2.Add Capacity constraint.
        self.routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # null capacity slack
            self.data['vehicle_capacities'],  # vehicle maximum capacities
            True,  # start cumul to zero
            'Capacity')

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

        # Solve the problem.
        self.solution = self.routing.SolveWithParameters(search_parameters)
    

        # Print solution on console.
        if self.solution:
            self.print_solution()
            return self.all_loop_route, self.demand_route
        else:
            print(self.solution)

In [10]:
time_matrix = np.load('../locate_call/강남역_경로매트릭스_예시(분).npy')
node = np.load('../locate_call/강남역_경로매트릭스_예시(분).npy')[:10,:10]
bus_name_lst = ['강남역','학동역','현대아파트','신당동.청구역','동묘앞','안암오거리','서초동삼성아파트','방배역','사당역13번출구','남성초등학교','언북중학교입구',
'세관앞','신구중학교','금옥초등학교.금호대우아파트','장충체육관앞','대광고등학교앞','서초역2번출구','청호나이스','방배동신동아아파트','서울고사거리','방배동래미안아파트','서울교통공사','구름다리','이수역','사당역']

In [11]:
my_sol = init_loop_vrp()

In [12]:
my_sol.get_loop(3, range(10), time_matrix)

Objective: 423856


([[0, 7, 8, 9, 6], [0, 3, 4, 5], [0, 2, 1]],
 [[0, 1, 1, 1, 1], [0, 1, 1, 1], [0, 2, 2]])